In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from networkx import NetworkXError
from tqdm import tqdm

from quapopt.additional_packages.ancillary_functions_usra import ancillary_functions as anf
from quapopt.data_analysis.data_handling import (COEFFICIENTS_TYPE,
                                                 COEFFICIENTS_DISTRIBUTION,
                                                 CoefficientsDistributionSpecifier,
                                                 HAMILTONIAN_MODELS)
from quapopt.hamiltonians.generators import build_hamiltonian_generator, ERDOS_RENYI_TYPES

### Generate random Hamiltonians and save them to files

* Similar to notebook A01 but we run it in the loop and also write Hamiltonians to file



In [ ]:
hamiltonian_model = HAMILTONIAN_MODELS.SherringtonKirkpatrick
full_noq_range = [125]

full_seeds_range = list(range(0, 3))

coefficients_type = COEFFICIENTS_TYPE.DISCRETE
coefficients_distribution = COEFFICIENTS_DISTRIBUTION.Uniform
coefficients_distribution_properties = {'low': -1, 'high': 1, 'step': 1}

coefficients_distribution_specifier = CoefficientsDistributionSpecifier(CoefficientsType=coefficients_type,
                                                                        CoefficientsDistributionName=coefficients_distribution,
                                                                        CoefficientsDistributionProperties=coefficients_distribution_properties)

arguments_dict_builder = {}

localities = None
erdos_renyi_type = None
#In some cases, we are required to specify whether to add local fields (localities = (1,2)) or just 2-body interactions (localities = (2,))
if hamiltonian_model in [HAMILTONIAN_MODELS.SherringtonKirkpatrick, ]:
    localities = (2,)
elif hamiltonian_model in [HAMILTONIAN_MODELS.RegularGraph]:
    localities = (2,)
elif hamiltonian_model in [HAMILTONIAN_MODELS.ErdosRenyi]:
    localities = (1, 2)
    erdos_renyi_type=ERDOS_RENYI_TYPES.Gnp
elif hamiltonian_model in [HAMILTONIAN_MODELS.MaxCut]:
    pass
elif hamiltonian_model in [HAMILTONIAN_MODELS.MAX2SAT]:
    pass
elif hamiltonian_model in [HAMILTONIAN_MODELS.Wishart]:
    pass

generator_cost_hamiltonian = build_hamiltonian_generator(hamiltonian_model=hamiltonian_model,
                                                         localities=localities,
                                                         coefficients_distribution_specifier=coefficients_distribution_specifier,
                                                         erdos_renyi_type=erdos_renyi_type)

print(full_noq_range)


In [ ]:
solve_hamiltonians = True

for number_of_qubits in tqdm(full_noq_range):
    anf.cool_print("NUMBER OF QUBITS", number_of_qubits)

    for seed_hamiltonian in full_seeds_range:
        print("INSTANCE:", seed_hamiltonian)



        arguments_instance = {'number_of_qubits': number_of_qubits,
                              'seed': seed_hamiltonian,
                              'read_from_drive_if_present': True}

        if hamiltonian_model in [HAMILTONIAN_MODELS.SherringtonKirkpatrick, ]:
            pass
        elif hamiltonian_model in [HAMILTONIAN_MODELS.RegularGraph]:
            graph_degree = 3
            arguments_instance['graph_degree'] = graph_degree
        elif hamiltonian_model in [HAMILTONIAN_MODELS.ErdosRenyi]:
            average_degree = number_of_qubits // 2
            #edge_probability = average_degree / (number_of_qubits - 1)
            edge_probability = 0.01

            arguments_instance['p_or_M'] = edge_probability
        elif hamiltonian_model in [HAMILTONIAN_MODELS.MaxCut]:
            average_degree = number_of_qubits // 2
            edge_probability = average_degree / (number_of_qubits - 1)
            arguments_instance['p_or_M'] = edge_probability
        elif hamiltonian_model in [HAMILTONIAN_MODELS.MAX2SAT]:
            clause_density = 1.1
            arguments_instance['clause_density'] = clause_density

        elif hamiltonian_model in [HAMILTONIAN_MODELS.WishartPlantedEnsemble]:
            pass

        try:
            cost_hamiltonian = generator_cost_hamiltonian.generate_instance(**arguments_instance)
        except NetworkXError as e:
            print("NetworkXError", e)
            continue

        if solve_hamiltonians:
            #This will write known solutions if they already exist in instance attributes (if not, it doesn't do anything)
            cost_hamiltonian.write_solutions_to_file()
            #This solving is required to get lowest energy state and highest energy state
            cost_hamiltonian.solve_hamiltonian(both_directions=True, #both directions means whether to also find highest energy state
                                               solver_kwargs={'solver_name': "BURER2002",
                                                              'solver_timeout': 10})

        ground_state_energy = cost_hamiltonian.ground_state_energy
        highest_energy = cost_hamiltonian.highest_energy

        cost_hamiltonian.write_to_file()



In [ ]:
print("Okay, then")